In [1]:
# Install dependencies
%pip install TikTokApi

%pip install nest_asyncio
%pip install tqdm
%pip install httpx
!python -m playwright install

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip list | grep TikTokApi

TikTokApi          6.2.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Store dict to json file
# create directory in path if not exist
import json
import os

def store_json(data, path, filename):
    if not os.path.exists(path):
        os.makedirs(path)
    with open(path + filename, 'w') as fp:
        json.dump(data, fp)

In [3]:
import httpx
import asyncio
import time

async def get_all_comments(video_id: str) -> list:
    url = "https://www.tiktok.com/api/comment/list/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/118.0",
        "Accept": "*/*",
        "Accept-Language": "de,en-US;q=0.7,en;q=0.3",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Pragma": "no-cache",
        "Cache-Control": "no-cache"
    }
    params = {
        "WebIdLastTime": "1697789782",          # Last time the WebId was used
        "aid": "1988",                          # Some form of application ID
        "app_language": "ja-JP",                # Application's language
        "app_name": "tiktok_web",               # Application's name
        "aweme_id": video_id,                   # Video's ID for which you are fetching comments
        "browser_language": "de",               # Browser's language
        "browser_name": "Mozilla",              # Browser's name
        "browser_online": "true",               # Whether the browser is online
        "browser_platform": "MacIntel",         # Browser's platform
        "browser_version": "5.0 (Macintosh)",   # Browser's version
        "channel": "tiktok_web",                # Channel (appears to be web-based in this context)
        "cookie_enabled": "true",               # Whether cookies are enabled
        "count": "20",                          # Number of comments to fetch per request
        "current_region": "JP",                 # Current region
        "cursor": "20",                          # Cursor (used for pagination)
        "device_id": "7291951547347551776",     # Device ID
        "device_platform": "web_pc",            # Device platform
        "enter_from": "tiktok_web",             # Entry source
        "focus_state": "true",                  # Some kind of focus state (maybe if the tab is active)
        "fromWeb": "1",                         # Indicates if the request is from web
        "from_page": "video",                   # The page from which the request is made
        "history_len": "6",                     # Some history length value
        "is_fullscreen": "false",               # Whether it's in fullscreen mode
        "is_non_personalized": "false",         # Whether the content is non-personalized
        "is_page_visible": "true",              # Whether the page is visible
        "os": "mac",                            # Operating system
        "priority_region": "DE",                # Priority region
        "referer": "",                          # Referring page (empty in this example)
        "region": "DE",                         # Region
        "screen_height": "900",                 # Screen height
        "screen_width": "1440",                 # Screen width
        "tz_name": "Europe/Berlin",             # Time zone name
        "webcast_language": "de-DE",            # Webcast language
        "msToken": "yd54NYSylgoRsnuvvI1q9PBa_s10lbJjzbHWve-tgG_Df8EATv36UypMmSOvx6YYbrOjc9Gt8s8DtOn55yvT8w1OUMX5CCTXsrWCxSjtCgv3bVfOGMhLkWbTBydBIg21mlNy",   # Some form of token
        "X-Bogus": "DFSzsIVuxo2ANnkatYQTA09WcBjf",   # Bogus header value
        "_signature": "_02B4Z6wo00001FLM12wAAIDAUszXblCurexSztPAAHGM4b"
    }


    
    all_comments = []
    
    async with httpx.AsyncClient() as client:
        while True:
            response = await client.get(url, params=params, headers=headers)
            print(response.text)
            print(type(response.text))
            # convert the response to JSON
            if response.text == "": break
            data = json.loads(response.text)
            

            all_comments.extend(data.get('comments', []))
            if not data.get("has_more"):
                break
            params["cursor"] = data.get("cursor")

    return all_comments

# To run the function
# loop = asyncio.get_event_loop()
# start = time.time()
# comments = loop.run_until_complete(get_all_comments("7289448316167933217"))
# end = time.time()
# print(f"Time taken: {end-start} seconds")
# print(comments)
# print(len(comments))


In [1]:
from TikTokApi import TikTokApi
import asyncio
import time
import nest_asyncio
nest_asyncio.apply()

ms_token_1 = "d6MAjHLjRT7er65L_nZKY_K1c0er_5GmW8wKqHEz37iQrgQWoehXoqGbaZx9LCbc2-ZKk1mMqCVPabBR1-IoK8entlFcTw6Mq2lTO4wy4Rt5-LSIrGHMiBQF6S123EaTuAyN" 


async def video_comments(username: str, video_id: str, comment_count: int):
    async with TikTokApi() as api:
        start_time = time.time()
        await api.create_sessions(num_sessions=5)
        stop_time = time.time()
        elapsed_time = stop_time - start_time
        print(f"Created sessions in {elapsed_time} seconds")

        total = 0
        max_batch_size = 20

        try:
            while total < comment_count:
                batch_size = max_batch_size
                if total + max_batch_size > comment_count:
                    batch_size = comment_count - total

                print(f"Staring batch {total} to {total + batch_size} of {comment_count}")
                start_time = time.time()
                async for comment in api.video(id= video_id).comments(count=batch_size, cursor=total):
                    store_json(comment.as_dict, './data/' + username + "/comments/" + video_id + "/", comment.id + '.json')
                stop_time = time.time()
                elapsed_time = stop_time - start_time
                print(f"Finished batch {total} to {total + batch_size} of {comment_count} in {elapsed_time} seconds")
                total += max_batch_size
            
        except Exception as e:
                print(e)
        finally:
            await api.close_sessions()

async def user_videos(username: str, num_videos: int):
    async with TikTokApi() as api:
        await api.create_sessions(ms_tokens=[ms_token_1], num_sessions=5)
        user = api.user(username=username)
        
        total = 0
        cursor = 0
        max_batch_size = 35
        while total < num_videos:
            batch_size = max_batch_size
            if total + max_batch_size > num_videos:
                batch_size = num_videos - total

            print(f"Staring batch {total} to {total + batch_size}")
            async for video in user.videos(count=batch_size, cursor=cursor):
                store_json(video.as_dict, './data/' + username + "/videos/", video.id + '.json')
                cursor = video.as_dict["createTime"] * 1000
            print(f"Finished batch {total} to {total + batch_size}")
            total += max_batch_size

        
        await api.close_sessions()

async def user_info(username: str):
    async with TikTokApi() as api:
        await api.create_sessions(ms_tokens=[ms_token_1], num_sessions=5)
        user = api.user(username=username)
        info = await user.info()
        store_json(info, './data/' + username + "/", username + '.json')
        await api.close_sessions()


        
usernames = ["afdfraktionimbundestag", "csuauftiktok", "fdpbt", "linksfraktion", "spdbt"]




# asyncio.run(user_info("csuauftiktok"))

asyncio.run(user_videos('afdfraktionimbundestag', 216))
# asyncio.run(user_info('spdbt'))

# asyncio.run(video_comments('spdbt', '6962453957851630853', 78))


Staring batch 0 to 35


EmptyResponseException: None -> TikTok returned an empty response

In [6]:
# load all json files in a directory
import glob
import json

def load_json(path):
    json_files = glob.glob(path + "*.json")
    data = []
    for file in json_files:
        with open(file) as json_file:
            data.append(json.load(json_file))
    return data

def check_if_comments_exist(username: str, video_id: str):
    if not os.path.exists('./data/' + username + "/videos/"+ video_id + ".json"):
        return False
    with open('./data/' + username + "/videos/"+ video_id + ".json", 'r') as file:
        data = json.load(file)
    comment_count = data["stats"]["commentCount"]
    comments = glob.glob('./data/' + username + "/comments/"+ video_id + "/*.json")
    # check if at least 50% of comments exist
    return len(comments) > comment_count * 0.5


async def load_all_comments(username: str):
    videos = load_json('./data/' + username + "/videos/")
    videos.sort(key=lambda x: x["createTime"])

    for video in videos:
        print("Percent complete: " + str(int((videos.index(video) / len(videos))*1000)))
        video_id = video["id"]
        if check_if_comments_exist(username, video_id):
            print(f"Video {video['id']} already has comments")
            continue

        comment_count = video["stats"]["commentCount"]
        try: 
            await video_comments(username, video_id, comment_count)
        except Exception as e:
            print(f"Video {video_id} failed to load comments with error: {e}")




# asyncio.run(load_all_comments('spdbt'))


In [7]:
# find element by id


In [12]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time
import subprocess
import glob
import random


def start_browser():
    
    # Define the proxy settings
    PROXY_HOST = "localhost"
    PROXY_PORT = 8080  # Default port for mitmproxy

    options = Options()

    # Configure Firefox to use the proxy
    options.set_preference("network.proxy.type", 1)
    options.set_preference("network.proxy.http", PROXY_HOST)
    options.set_preference("network.proxy.http_port", PROXY_PORT)
    options.set_preference("network.proxy.ssl", PROXY_HOST)
    options.set_preference("network.proxy.ssl_port", PROXY_PORT)

    # Disable Firefox from checking the validity of the SSL certificate
    options.set_preference("security.enterprise_roots.enabled", True)

    # Start browser
    browser = webdriver.Firefox(options=options)
    browser.get("https://www.tiktok.com/@spdbt/video/6962453957851630853")

    # wait for key input 
    input("Press Enter to continue...")
    return browser

def start_mitmdump():
    # Start mitmdump with the script
    cmd = ["mitmdump", "-s", "dump_flow.py"]
    process = subprocess.Popen(cmd)
    return process

def count_json_files(path):
    json_files = glob.glob(path + "/*.json")
    return len(json_files)

def auto_scroll(browser, username, id):
    base_path = './data/{}/comments/{}'.format(username,id)
    json_files = count_json_files(base_path)
    
    while True:
        for i in range(0, 3):
            try:
                while True:
                    browser.find_element(By.ID,"captcha-verify-image")
                    time.sleep(1)
            except:
                print("No captcha found")
            # Scroll down to the bottom
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait for a random time to emulate human behavior
            time.sleep(random.uniform(1, 2))  # Wait between 1.5 to 3.5 seconds
            # Check the number of JSON files
            new_json_files = count_json_files(base_path)
            
            if new_json_files != json_files:
                    break
        if new_json_files == json_files:
            break
        # Update the counter
        json_files = new_json_files


def close(browser, process):
    print("Done")
    browser.quit()
    # Once done, terminate mitmdump
    process.terminate()

def load_all_comments(username: str):
    process = start_mitmdump()
    browser = start_browser()

    videos = load_json('./data/' + username + "/videos/")
    videos.sort(key=lambda x: x["createTime"])

    for video in videos:
        # skip if comments already exist
        if check_if_comments_exist(username, video["id"]):
            print("Video already has comments")
            continue
        print("*************************************************************************************************")
        print("Percent complete: " + str(int((videos.index(video) / len(videos))*1000)))
        print("*************************************************************************************************")
        video_id = video["id"]
        browser.get(f"https://www.tiktok.com/@{username}/video/" + video_id)
        time.sleep(3)
        auto_scroll(browser, username, video_id)

    close(browser, process)

# load_all_comments('spdbt')


The geckodriver version (0.31.0) detected in PATH at /opt/homebrew/bin/geckodriver might not be compatible with the detected firefox version (118.0.2); currently, geckodriver 0.33.0 is recommended for firefox 118.*, so it is advised to delete the driver in PATH and retry
Error logged during startup, exiting...


[16:33:41.545] Loading script dump_flow.py
[16:33:41.548] [Errno 48] HTTP(S) proxy failed to listen on *:8080 with [Errno 48] error while attempting to bind on address ('0.0.0.0', 8080): address already in use
Try specifying a different port by using `--mode regular@8082`.
[16:33:43.073][127.0.0.1:60754] client connect
[16:33:43.245][127.0.0.1:60754] server connect location.services.mozilla.com:443 (54.213.186.101:443)
[16:33:43.742][127.0.0.1:60756] client connect
[16:33:43.745][127.0.0.1:60757] client connect
[16:33:43.819][127.0.0.1:60756] server connect firefox.settings.services.mozilla.com:443 (34.149.100.209:443)
[16:33:43.820][127.0.0.1:60757] server connect detectportal.firefox.com:80 (34.107.221.82:80)
127.0.0.1:60757: GET http://detectportal.firefox.com/canonical.html
              << 200 OK 90b
[16:33:43.881][127.0.0.1:60760] client connect
[16:33:43.882][[::1]:60761] client connect
[16:33:43.913][127.0.0.1:60760] server connect detectportal.firefox.com:80 (34.107.221.82:80)

In [1]:
# count the total number of comments
import glob
import json

def count_comments(username: str):
    json_files = glob.glob('./data/' + username + "/comments/*/*.json")
    return len(json_files)

print(count_comments('spdbt'))

10866


In [ ]:
Event in the middle 

# politics

# Analyse sent

# 

